In [212]:
import pandas as pd
import numpy as np
from scipy.stats import skew
import pickle
from pickle import dump
from pickle import load
from scipy import stats
from scipy.stats import norm
import seaborn as sns
import matplotlib.pyplot as plt
import category_encoders as ce
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
import xgboost
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.impute import SimpleImputer
#from mlmachine.features.preprocessing import GroupbyImputer
from sktutor.preprocessing import GroupByImputer
from sklearn.metrics import mean_squared_error
from bayes_opt import BayesianOptimization

# to visualise al the columns in the dataframe
pd.pandas.set_option("display.max_columns", None)
pd.pandas.set_option("display.max_rows", None)

dataset_train=pd.read_csv("train.csv")

##preprocessing done based on insights from EDA analysis

In [ ]:
train=pd.read_csv("train_processed.csv")
Y=train['SalePrice']
X=train.drop('SalePrice',axis=1)

##convert to matrix fromat for XGBoost CV
dtrain = xgboost.DMatrix(X, Y)

In [197]:
import xgboost as xgb
xgb_regressor=xgb.XGBRegressor()

###function to evaluate the hyper paramters by CV approach
def xgb_evaluate(max_depth, gamma,reg_alpha, eta, n_estimators, min_child_weight, subsample, num_boost_round, colsample_bytree):
    params = {
              'booster' : 'gbtree',
              'max_depth' : int(max_depth),
              'gamma' : gamma,
              'n_estimators' : n_estimators,
              'eta' : eta,
              'eval_metric': 'rmse',
              'subsample' : subsample,
              'colsample_bytree' : colsample_bytree,
              'min_child_weight' : min_child_weight,
              'num_boost_round':  num_boost_round,
              'reg_alpha':reg_alpha,
              'seed' : 1001
              }
    cv_result = xgb.cv(params,
                    dtrain,
                    num_boost_round = 100,
                    nfold = 5,
                    verbose_eval = 10,
                    early_stopping_rounds = 10,
                    metrics = 'rmse',
                    show_stdv = True
               )
    return -1.0 * cv_result['test-rmse-mean'].iloc[-1]

In [ ]:

###bayesian optimizer to find best params in the defined space
xgb_bo = BayesianOptimization(xgb_evaluate, {
    'eta':(0.01,0.1),
    'max_depth': (2, 5), 
    'gamma': (0.001, 10.0),
    'min_child_weight': (0, 20),
    'subsample': (0.4, 1.0),
    'num_boost_round':(20,100),
    'colsample_bytree' :(0.2, 1.0),
    'reg_alpha': (0.05, 1),
    'n_estimators' :(800,1300) 
    })

xgb_bo.maximize(init_points=20, n_iter=100, acq='ei')

In [204]:
#Extracting the best parameters
#params = xgb_bo.max['params']
#print(params)
## after iterating throudh different hyperparam space, the below params have given the least RMSE on test set
###model can be further improved by fine tuning regularization parameters
params = {'colsample_bytree': 0.3981871741945951,
 'eta': 0.09810724458093252,
 'gamma': 1.5506850241079462,
 'max_depth': 5,
 'min_child_weight': 0.09884157087012513,
 'n_estimators': 816,
 'num_boost_round': 92.80247852525846,
 'reg_alpha': 0,
 'subsample': 0.6949551355023206,}

In [ ]:
#Converting the max_depth and n_estimator values from float to int
params['max_depth']= int(params['max_depth'])
params['n_estimators']= int(params['n_estimators'])
xgb_tuned = xgb.XGBRegressor(**params).fit(X, Y)

In [ ]:
##load processed test data
dataset_test=pd.read_csv("test_processed.csv")
test = dataset_test.drop(['Id'],axis=1)
y_test = xgb_tuned.predict(test)
y_train = xgb_tuned.predict(X)
print(y_test)
print(y_train)
#5,10:10,20, 10:100,20:100, 20:100  

In [ ]:
#print("Validation RMSE",np.sqrt(mean_squared_error(np.log(Y_val),np.log(ypred))))  

print("Training RMSE",np.sqrt(mean_squared_error(np.log(y_train),np.log(Y))))

In [211]:

dataset_test=pd.read_csv('test.csv',index_col='Id')
submission = pd.DataFrame({'Id': dataset_test.index, 'SalePrice': y_test})
submission.to_csv('submission_bopt_best.csv', index=False)

#Test RMSE: 0.12838 on kaggle,  standing at 848 position (top 20%). Further hyper-paramter tuning can improve the model 